In [1]:
# Algorithm:
# consider the bounded box (minx, miny) and (maxx, maxy)
# find the manhattan distances for each location within the box.
# points there are tagged on the boundary are infinite, others not.

function boundary(points)
    minx, maxx = extrema(point[1] for point in points)
    miny, maxy = extrema(point[2] for point in points)
    return minx, miny, maxx, maxy
end

function manhattan_distance(p, q)
    return abs(p[1] - q[1]) + abs(p[2] - q[2])
end

function has_multiple_nearest_neighbor(distances)
    count(x == minimum(distances) for x in distances) > 1
end

function nearest_point(point, candidate_points)
    distances = [manhattan_distance(point, from_point) for from_point in candidate_points]
    has_multiple_nearest_neighbor(distances) ? 0 : findmin(distances)[2]
end

function create_map(candidate_points)
    minx, miny, maxx, maxy = boundary(candidate_points)
    # note: some wasted memory since minx/miny isn't zero
    distance_map = zeros(Int, maxy, maxx)
    for y in miny:maxy
        for x in minx:maxx
           distance_map[y, x] = nearest_point([x,y], candidate_points)
        end
    end
    return distance_map
end

function eliminated_ones(distance_map)
    # note: zero is included in the result intentionally
    return unique(vcat(
            distance_map[1,:], distance_map[end,:], 
                distance_map[:,1], distance_map[:,end]))
end

function areas_by_point(distance_map)
    # Create a (point => area) dictionary
    # note: is there a better way to do this?
    M, N = size(distance_map)
    distance_array = reshape(distance_map, (M*N,))
    distance_count = countmap(distance_array)
    # Remove ones that are tagged on the boundary
    for e in eliminated_ones(distance_map)
        delete!(distance_count, e)
    end
    return distance_count
end

function max_area(areas)
    let point_index = argmax(areas)
        return point_index, areas[point_index]
    end
end

using StatsBase
points = [parse.(Int, split(L, ", ")) for L ∈ readlines("input6")]
create_map(points) |> areas_by_point |> max_area

(28, 4143)

In [2]:
# part 2
function total_distance(point, candidate_points)
    return sum(manhattan_distance(point, from_point) for from_point in candidate_points)
end

function is_safe(point, candidate_points; safe_distance = 10000)
    return total_distance(point, candidate_points) < safe_distance
end

function create_safety_map(candidate_points; kwargs...)
    minx, miny, maxx, maxy = boundary(candidate_points)
    # note: some wasted memory since minx/miny isn't zero
    safety_map = zeros(Int, maxy, maxx)
    for y in miny:maxy
        for x in minx:maxx
           safety_map[y, x] = is_safe([x,y], candidate_points; kwargs...) ? 1 : 0
        end
    end
    return safety_map
end

create_safety_map(points) |> sum

35039

In [3]:
# testing
test_points = [[1,1], [1,6], [8,3], [3,4], [5,5], [8,9]]
@show test_box = boundary(test_points)
@show manhattan_distance([1,1], [1,1])
@show manhattan_distance([1,1], [2,1])
@show manhattan_distance([1,1], [2,2])
@show nearest_point([3,2], test_points)
@show create_map(test_points)
@show create_map(test_points) |> eliminated_ones
create_safety_map(test_points, safe_distance = 32) 

# for y in 1:9
#     for x in 1:8
#         p = nearest_point([x,y], test_points)
#         print(p)
#     end
#     println()
# end
# println("""
# Aaaa.ccc
# aaddeccc
# adddeccC
# .dDdeecc
# b.deEeec
# Bb.eeee.
# bb.eeeff
# bb.eefff
# bb.ffffF
# """)

test_box = boundary(test_points) = (1, 1, 8, 9)
manhattan_distance([1, 1], [1, 1]) = 0
manhattan_distance([1, 1], [2, 1]) = 1
manhattan_distance([1, 1], [2, 2]) = 2
nearest_point([3, 2], test_points) = 4
create_map(test_points) = [1 1 1 1 0 3 3 3; 1 1 4 4 5 3 3 3; 1 4 4 4 5 3 3 3; 0 4 4 4 5 5 3 3; 2 0 4 5 5 5 5 3; 2 2 0 5 5 5 5 0; 2 2 0 5 5 5 6 6; 2 2 0 5 5 6 6 6; 2 2 0 6 6 6 6 6]
create_map(test_points) |> eliminated_ones = [1, 0, 3, 2, 6]


9×8 Array{Int64,2}:
 0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0
 0  0  1  1  1  0  0  0
 0  1  1  1  1  1  0  0
 0  1  1  1  1  1  0  0
 0  0  1  1  1  0  0  0
 0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0

In [5]:
?countmap

search: countmap



```
countmap(x; alg = :auto)
```

Return a dictionary mapping each unique value in `x` to its number of occurrences.

  * `:auto` (default): if `StatsBase.radixsort_safe(eltype(x)) == true` then use                    `:radixsort`, otherwise use `:dict`.
  * `:radixsort`:      if `radixsort_safe(eltype(x)) == true` then use the                    [radix sort](https://en.wikipedia.org/wiki/Radix_sort)                    algorithm to sort the input vector which will generally lead to                    shorter running time. However the radix sort algorithm creates a                    copy of the input vector and hence uses more RAM. Choose `:dict`                    if the amount of available RAM is a limitation.
  * `:dict`:           use `Dict`-based method which is generally slower but uses less                    RAM and is safe for any data type.
